In [1]:
import numpy as np
from scipy.interpolate import UnivariateSpline
from astrotools.spectrum import Spectrum
from astrotools.filters import Filters, FILTER_DIR


def func(wav, flux, params, flt):
    npars = params.size

    # spline control points
    splineX = np.zeros(npars+2)
    splineY = np.zeros(npars+2)

    # Set central control points
    for i in np.arange(0, npars):
        splineX[i+1] = flt[i].wav
        splineY[i+1] = params[i]

    # Set first control point
    splineX[0] = np.min(wav) - 100
    splineY[0] = splineX[0] * ((splineY[1] - splineY[2]) / (splineX[1] - splineX[2]))
    splineY[0] += ((splineX[1] * splineY[2] - splineY[1] * splineX[2]) /
                   (splineX[1] - splineX[2]))

    # Set last control point
    splineX[npars+1] =  np.max(wav) + 100
    splineY[npars+1] = splineX[npars+1] * ((splineY[npars-1] - splineY[npars]) /
                       (splineX[npars-1] - splineX[npars]))
    splineY[npars+1] += ((splineX[npars-1] * splineY[npars] - splineY[npars-1] *
                         splineX[npars]) / (splineX[npars-1] - splineX[npars]))

    # initialise gsl spline
    spl = UnivariateSpline(splineX, splineY)
    
    # mangle the spectra
    return flux * spl(wav)

def residual(lc_data, flt):
    mangledSpec = func(wav, flux, params, flt)
    res = np.zeros(lc_data.size);

    for i in np.arange(res.size):
        res[i] = (lc_data[i].flux - flt.flux(mangledSpec, lc_data[i].flt)) / lc_data[i].flux_err
 
    return res

In [2]:
flt = Filters(load_all=True)
spec = Spectrum(file_name='/Users/szymon/Projects/CoCo/data/spec/SN2011dh/2011dh_-12.68.txt')

In [3]:
spec.synthesis_photometry(['Bessell_V'], flt)

array([  1.63280822e-17])